In [ ]:
# Clean up old CSV files
!rm actions_*.csv

In [ ]:
!time python battle.py

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def load(p):
    df = pd.read_csv(p)
    df['csv'] = str(p)
    return df

df = pd.concat([load(x) for x in Path().glob("actions_*.csv") if x.stat().st_size > 1000], ignore_index=True)
df

How long do battles typically last, as a function of epoch?

In [ ]:
x = df.groupby(['epoch', 'encounter', 'csv'])['round'].max().groupby('epoch')
y = x.quantile(0.1); plt.plot(y.index, y)
y = x.quantile(0.5); plt.plot(y.index, y)
y = x.quantile(0.9); plt.plot(y.index, y)

How often do we choose to Dodge, as a function of time?

In [ ]:
x = df.query('actor == "Hero" and action == "Dodge"').groupby('epoch').count()
plt.plot(x.index, x.encounter)

How often do we use a healing potion?

In [ ]:
x = df.query('actor == "Hero" and raw_hp > 0').groupby('epoch')['encounter'].count() / 1000
plt.plot(x.index, x)

Do we use our healing potions more effectively as time goes on?

In [ ]:
# Mean effect?
x = df.query('raw_hp > 0').groupby('epoch')['obs_hp'].mean()
plt.plot(x.index, x)
# How often do we use it when we're uninjured?  Not often, it's forbidden.
# x = df.query('raw_hp > 0 and obs_hp == 0').groupby('epoch').count()
# plt.plot(x.index, x.obs_hp)

How often do we target the weakest enemy?

In [ ]:
x = df.query('actor == "Hero" and action == "long sword"').groupby(['epoch']).mean()
plt.plot(x.index, x.t_weakest)

How often do we switch targets within an encounter?

In [ ]:
x = df.query('actor == "Hero" and action == "long sword"').copy()
#x['round2'] = x.groupby(['epoch', 'encounter', 'csv'])['round'].rank().astype('int')
# Convert target to numerical values
x['tgt'] = pd.Categorical(x.target).codes
# Detect changes in target
x['change_tgt'] = (x.groupby(['epoch', 'encounter', 'csv'])['tgt'].diff() != 0)
x.head()
#dir(x.tgt.cat)

In [ ]:
# Minimum is 2 (for a win):  initial attack, and switch to second target
y = x.groupby('epoch')['change_tgt'].sum() / 1000
plt.plot(y.index, y)

Which enemy do we attack first?

In [ ]:
y = x.query('round == 0').groupby('epoch')['tgt'].mean()
plt.plot(y.index, y)